In [ ]:
import tableauserverclient as TSC
from tableauhyperapi import HyperProcess, Telemetry, \
    Connection, CreateMode, \
    NOT_NULLABLE, NULLABLE, SqlType, TableDefinition, \
    Inserter, \
    escape_name, escape_string_literal, \
    HyperException,TableName

from pathlib import Path
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils.querying import get_projects_dataframe
import pandas as pd

# Login to Tableau Online and open a session
# Change the parameter as per your server
tableau_server_config = {
        'my_env': {
                'server': 'https://10ax.online.tableau.com',
                'api_version': '3.12',
                'personal_access_token_name':'vivek',
                'personal_access_token_secret':'',
                'site_name': '',
                'site_url': ''
        }
}
conn = TableauServerConnection(tableau_server_config, env='my_env')
conn.sign_in()

# select csv file from local drive
PATH_TO_CSV="./covid_india.csv"
PATH_TO_HYPER = "Covid.hyper"

df=pd.read_csv(PATH_TO_CSV)
df.head()

with HyperProcess(Telemetry.SEND_USAGE_DATA_TO_TABLEAU,'HyperExample') as hyper:
    connection_parameters = {"lc_time": "en_US"}
    with Connection(endpoint=hyper.endpoint,
                    database=PATH_TO_HYPER,
                    create_mode=CreateMode.CREATE_AND_REPLACE) as connection:
        
        connection.catalog.create_schema('Extract')
        print("The connection to the Hyper file is open.")
        schema=TableDefinition(table_name=TableName('Extract','Extract'), 
            columns=[
                TableDefinition.Column("S. No.",SqlType.int(),NOT_NULLABLE),
                TableDefinition.Column("Name of State UT",SqlType.text(),NOT_NULLABLE),
                TableDefinition.Column("Active Cases",SqlType.big_int(),NOT_NULLABLE),
                TableDefinition.Column("Cured_Discharged_Migrated",SqlType.big_int(),NOT_NULLABLE),
                TableDefinition.Column("Deaths",SqlType.int(),NOT_NULLABLE),
                TableDefinition.Column("Total Confirmed cases",SqlType.big_int(),NOT_NULLABLE)
            ])
        print("The table is defined.")
        
        connection.catalog.create_table(schema)
        with Inserter(connection, schema) as inserter:
            for index, row in df.iterrows():
                inserter.add_row(row)
            inserter.execute()

        print("The connection to the Hyper file has been closed.")
    print("The Hyper process has been shut down.")

#
projects_df = get_projects_dataframe(conn)
print(projects_df[['name', 'id']])
    
response = conn.publish_data_source(
    datasource_file_path=PATH_TO_HYPER,
    datasource_name='Hyper_sample',
    project_id='83d0bbe6-ce59-413e-a26e-193bef8cc0ca'
    )
print("Hyper Connection puboished to Server")
conn.sign_out()